# Pathology prediction (lung lesion)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the lung lesion pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the Lung Lesion pathology is : (N = 930)


#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.LUNG_LESION, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="LungLesion", experiment_name="CP_LungLesion", filepath="./results/lunglesion")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-26 10:03:33,478	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=396015) Outer fold 1
(run_fold pid=396015) Configuring PyCaret for outer fold 1
(run_fold pid=396009) Outer fold 2 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Processing:  75%|███████▌  | 3/4 [00:27<00:10, 10.12s/it] [repeated 4x across cluster]
                                                         
Processing:  75%|███████▌  | 3/4 [00:28<00:10, 10.56s/it] [repeated 3x across cluster]


(run_fold pid=396014)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=396014) Fold                                                          
(run_fold pid=396014) 0       0.9748  0.9912     1.0  0.9744  0.9870  0.5609  0.6243
(run_fold pid=396014) 1       0.9664  0.5982     1.0  0.9661  0.9828  0.3239  0.4396
(run_fold pid=396014) 2       0.9664  0.8491     1.0  0.9661  0.9828  0.3239  0.4396
(run_fold pid=396014) 3       0.9664  0.6754     1.0  0.9661  0.9828  0.3239  0.4396
(run_fold pid=396014) 4       0.9664  0.8230     1.0  0.9658  0.9826  0.4871  0.5674
(run_fold pid=396014) Mean    0.9681  0.7874     1.0  0.9677  0.9836  0.4039  0.5021
(run_fold pid=396014) Std     0.0034  0.1378     0.0  0.0033  0.0017  0.1008  0.0786
(run_fold pid=396009) Configuring PyCaret for outer fold 2 [repeated 3x across cluster]
(run_fold pid=396014) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=396015) 0       0.9748  0.9140     

Processing:   0%|          | 0/4 [00:00<?, ?it/s]
                                                          [repeated 3x across cluster]
Processing:  25%|██▌       | 1/4 [00:00<00:02,  1.47it/s]


(run_fold pid=396014) Transformation Pipeline and Model Successfully Saved
(run_fold pid=396014)                        Model  Accuracy     AUC  ...     F1  Kappa  MCC
(run_fold pid=396014) 0  Extreme Gradient Boosting     0.957  0.7781  ...  0.978    0.0  0.0
(run_fold pid=396014) 
(run_fold pid=396014) [1 rows x 8 columns]
(run_fold pid=396009) Transformation Pipeline and Model Successfully Saved
(run_fold pid=396009)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=396009) 0  Extreme Gradient Boosting    0.9677  0.9256  ...  0.9834  0.3895  0.4918
(run_fold pid=396009) 
(run_fold pid=396009) [1 rows x 8 columns]


Processing:  75%|███████▌  | 3/4 [00:26<00:09,  9.83s/it]


(run_fold pid=396015)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=396015) Fold                                                          
(run_fold pid=396015) 0       0.9664  0.8263     1.0  0.9661  0.9828  0.3239  0.4396
(run_fold pid=396015) 1       0.9664  0.7684     1.0  0.9661  0.9828  0.3239  0.4396
(run_fold pid=396015) 2       0.9664  0.8368     1.0  0.9661  0.9828  0.3239  0.4396
(run_fold pid=396015) 3       0.9580  0.9070     1.0  0.9580  0.9785  0.0000  0.0000
(run_fold pid=396015) 4       0.9580  0.9307     1.0  0.9576  0.9784  0.2753  0.3995
(run_fold pid=396015) Mean    0.9630  0.8539     1.0  0.9628  0.9810  0.2494  0.3436
(run_fold pid=396015) Std     0.0041  0.0584     0.0  0.0041  0.0021  0.1261  0.1725
(run_fold pid=396015) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=396012) Transformation Pipeline and Model Successfully Saved
(run_fold pid=396012)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=396012) 0  Extreme Gradient Boosting    0.9624  0.8729  ...  0.9807  0.2147  0.3468
(run_fold pid=396012) 
(run_fold pid=396012) [1 rows x 8 columns]
(run_fold pid=396015) Transformation Pipeline and Model Successfully Saved
(run_fold pid=396015)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=396015) 0  Extreme Gradient Boosting    0.9624  0.7866  ...  0.9806  0.3522  0.4624
(run_fold pid=396015) 
(run_fold pid=396015) [1 rows x 8 columns]
Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.96976  0.004601
1       AUC  0.88772  0.069591
2    Recall  1.00000  0.000000
3     Prec.  0.96942  0.004546
4        F1  0.98448  0.002300
5     Kappa  0.41870  0.129810
6       MCC  0.51348  0.101164
